## Pytorch Practice- CIFAR 10 data

In [68]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import numpy as np
import pickle
import os
from PIL import Image
import random
import time
import torchvision

cuda_available = torch.cuda.is_available()

In [22]:
import numpy as np
import pickle
import os
from PIL import Image
import random
import time

#### Create Resnet Block

In [128]:
class MyResblock(nn.Module):
    def __init__(self, in_ch,out_ch,stride=1):
        
        super(MyResblock,self).__init__()
        # Conv layer 1
        self.conv1=nn.Conv2d(
            in_channels=in_ch,out_channels=out_ch,kernel_size=3,padding=1,stride=stride,bias=False
        )
        self.bn1=nn.BatchNorm2d(out_ch)
        
        # Conv Layer 2
        self.conv2=nn.Conv2d(
            in_channels=out_ch,out_channels=out_ch,kernel_size=3,padding=1,stride=1,bias=False
        )
        self.bn2=nn.BatchNorm2d(out_ch)
        
        self.short_cut=nn.Sequential()
        if stride!=1 or in_ch!=out_ch:
            self.short_cut=nn.Sequential(
            nn.Conv2d(
                in_channels=in_ch,out_channels=out_ch,kernel_size=1,stride=stride,bias= False
            ),
            nn.BatchNorm2d(out_ch)
            )
        
    def forward(self,x):
        out=nn.ReLU()(self.bn1(self.conv1(x)))
        out=self.bn2(self.conv2(out))
        out+=self.short_cut(x)
        out=nn.ReLU()(out)
        return out
    

###### Create main network

In [132]:
class MyResnet(nn.Module):
    def __init__(self,num_classes=10):
        super(MyResnet, self).__init__()
        
        # first convolution
        self.conv1=nn.Conv2d(
            in_channels=3,out_channels=64,stride=1,padding=1,bias=False, kernel_size= 3
        )
        self.bn1=nn.BatchNorm2d(64) #out channels from previous layer
        print('first conv done')
        
        self.block1=self._create_res_blocks(64,64,stride=1)
        self.block2=self._create_res_blocks(64,128,stride=2)
        self.block3=self._create_res_blocks(128,256,stride=2)
        self.block4=self._create_res_blocks(256,512,stride=2)
        self.linear=nn.Linear(512,num_classes)
        
    def _create_res_blocks(self,in_ch,out_ch,stride):
            return nn.Sequential(
                MyResblock(in_ch,out_ch,stride),
                MyResblock(out_ch,out_ch,1)
            )
        
    def forward(self,x):
            out=nn.ReLU()(self.bn1(self.conv1(x)))
            out=self.block1(out)
            out=self.block2(out)
            out=self.block3(out)
            out=self.block4(out)
            out=nn.AvgPool2d(4)(out)
            out=out.view(out.size(0), -1)
            out=self.linear(out)
            return out

In [121]:
# Read Data Cifar-10
data_dir='C:/Users/Gaurav/data/cifar/cifar/'

In [122]:
with open(data_dir+'labels.txt') as label_file:
    labels=label_file.read().split()
    label_mapping=dict(zip(labels,list(range(len(labels)))))

In [123]:
label_mapping

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [28]:
# Process the images
def preprocess(image):
    image = np.array(image)
    
    if random.random() > 0.5:
        image = image[::-1,:,:]
    
    cifar_mean = np.array([0.4914, 0.4822, 0.4465]).reshape(1,1,-1)
    cifar_std  = np.array([0.2023, 0.1994, 0.2010]).reshape(1,1,-1)
    image = (image - cifar_mean) / cifar_std
    
    image = image.transpose(2,1,0)
    return image

In [81]:
# Create a custom dataset subclass to read the images
# Dataset class is provided by Pytorch to index into our data

class MyCustomDataset(torch.utils.data.Dataset):
    def __init__(self,data_dir,data_size=0,transforms=None):
        files=os.listdir(data_dir)
        files=[os.path.join(data_dir,x) for x in files]
        
        if data_size<0 or data_size>len(files):
            assert ('Data size should be between 0 to max files in the directory')
        
        if data_size==0:
            data_size=len(files)
        
        self.data_size=data_size
        self.files= random.sample(files,self.data_size)
        self.transforms=transforms
    
    def __len__(self):
        return self.data_size
    
    def __getitem__(self,idx):
        image_addr=self.files[idx]
        image=Image.open(image_addr)
        image=preprocess(image)
        label_name=image_addr[:-4].split("_")[-1]
        label=label_mapping[label_name]
        
        image=image.astype(np.float32)
        
        if self.transforms:
            image=self.transforms(image)
        
        return image,label

In [136]:
# Create Dataloader and the Dataset

trainset = MyCustomDataset(data_dir+'train')
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 64 , shuffle= True , num_workers= 0)

testset = MyCustomDataset(data_dir+'test')
testloader = torch.utils.data.DataLoader(trainset, batch_size= 64 , shuffle= True , num_workers= 0)

In [105]:
# select GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [137]:
clf = MyResnet()
clf.to(device)

first conv done


MyResnet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Sequential(
    (0): MyResblock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (short_cut): Sequential()
    )
    (1): MyResblock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

In [125]:
# Training and Evaluation

In [126]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(clf.parameters(),lr=0.1, momentum=0.9,weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones = [150,200], gamma = 0.1)

In [ ]:
# Creating the training loop

for epoch in range(10):
    losses=[]
    scheduler.step()
    
    #Train
    start_time = time.time()
    
    for batch_idx, (inputs,targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() # this is required so that our gradients don't accumulate after each epoch
        
        outputs = clf(inputs) # run the model on inputs
        loss = criterion(outputs,targets) # calc the Loss
        loss.backward() # Calc the gradients
        optimizer.step() # update the parameters 
        losses.append(loss.item())
        end_time = time.time()
        
        if batch_idx % 100 ==0:
            print('Batch Index: %d Loss : %.3f Time : %.3f seconds' %(batch_idx,np.mean(losses),end_time-start_time))
            
    # Eval Mode
    clf.eval()
    total = 0
    correct = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = clf(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += predicted.eq(targets.data).cpu().sum()

        print('Epoch : %d Test Acc : %.3f' % (epoch, 100.*correct/total))
        print('--------------------------------------------------------------')
    clf.train()  

Batch Index: 0 Loss : 2.337 Time : 6.059 seconds
Batch Index: 100 Loss : 2.318 Time : 620.555 seconds
Batch Index: 200 Loss : 2.315 Time : 1239.500 seconds
Batch Index: 300 Loss : 2.317 Time : 5511.076 seconds
Batch Index: 400 Loss : 2.317 Time : 9952.934 seconds
Batch Index: 500 Loss : 2.318 Time : 11643.863 seconds
